# App de consulta ao boletim escolar usando a biblioteca customtkinter

In [1]:
# Importanto bibliotecas
import requests
import customtkinter as ctk
from tksheet import Sheet
import pandas as pd

# caso não as tenha instalado
# basta executar os comandos:
# !pip install pandas
# !pip install requests
# !pip install customtkinter
# !pip install tksheet

## Lendo arquivo excel no drive

In [2]:
# Lista de URLs dos arquivos Excel
url_list = [
    'https://drive.google.com/uc?id=18UsC8LuVtqKQsiwi8tbiAq9ccOVA-mCI&export=download',
    'https://drive.google.com/uc?id=18UsC8LuVtqKQsiwi8tbiAq9ccOVA-mCI&export=download',
    'https://drive.google.com/uc?id=18UsC8LuVtqKQsiwi8tbiAq9ccOVA-mCI&export=download',
    'https://drive.google.com/uc?id=18UsC8LuVtqKQsiwi8tbiAq9ccOVA-mCI&export=download'
]

tbBim = {}  # Dicionário para armazenar os dados dos bimestres
for idx, url in enumerate(url_list, start=1):
    # Faz uma requisição HTTP para obter o conteúdo do arquivo Excel
    response = requests.get(url)
    content = response.content
    
    # Lê o arquivo Excel diretamente a partir do conteúdo baixado
    # Aqui, a função pd.read_excel é usada para ler o conteúdo do arquivo Excel sem a necessidade de salvar o arquivo localmente
    # A função sheet_name=None lê todas as abas do arquivo Excel
    # O parâmetro skiprows=1 indica que a primeira linha será ignorada
    # O parâmetro index_col=[0] define que a primeira coluna será usada como índice
    # O parâmetro header=[0, 1] indica que a primeira e segunda linha serão usadas como cabeçalho de colunas
    # O parâmetro engine='openpyxl' define o mecanismo de leitura como openpyxl
    tbNotas = pd.read_excel(content, sheet_name=None, skiprows=1, index_col=[0], header=[0, 1], engine='openpyxl')
    
    # Armazena os dados do bimestre no dicionário tbBim
    tbBim[f'{idx}ª Bimestre'] = tbNotas

## Criando um dicionário com a tabela de notas usando as turmas como chave.

In [3]:
bim_list = ["1ª Bimestre", "2ª Bimestre", "3ª Bimestre", "4ª Bimestre"]

# Obtém a lista de turmas a partir do primeiro bimestre
turma_list = list(tbBim["1ª Bimestre"].keys())[1:]

dicdf = {}
for turma in turma_list:
    # Criando uma lista com as tabelas de todos os 4 bimestres
    listdf_bim = list()
    for bim in bim_list:
        listdf_bim.append(pd.DataFrame(tbBim[bim][turma]))

    # Concatenando/juntando as tabelas em uma única com 4 grandes colunas 1ª, 2ª, 3ª e 4ª Bimestre
    df_bim = pd.concat(listdf_bim, axis=1, keys=bim_list)

    # Excluindo as duas últimas linhas e arredondando os valores para duas casas decimais
    df_bim.drop(df_bim.tail(2).index, axis=0, inplace=True)
    df_bim = df_bim.round(2)

    # Removendo a subcoluna 'MÉDIA' do nível 1
    df_bim = df_bim.drop('MÉDIA', axis=1, level=1)

    # Removendo a subcoluna 'TAXA DE ' do nível 1
    df_bim = df_bim.drop('TAXA DE ', axis=1, level=1)

    # Armazenando o DataFrame no dicionário usando a turma como chave
    dicdf[turma] = df_bim

## Criando App usando o customtkinter

In [6]:
# Basic parameters and initializations
# Supported modes : Light, Dark, System
ctk.set_appearance_mode("System")

# Supported themes : green, dark-blue, blue
ctk.set_default_color_theme("green")  

####################################################################
App = ctk.CTk() # Criando a aplicação/janela principal
####################################################################

# Define as opções da janela para maximizar
App.attributes('-fullscreen', True)
App.title("App de Gerenciamento de Notas") # Título da janela

################ FUNÇÔES #############################################
def selListDiscEstud(turma, Bim):
    global df
    df = dicdf[turma]
    
    if Bim == "Todos":
        # Se for selecionado "Todos", definir Bim como a lista completa dos bimestres
        Bim = bim_list[:]
        
        # Lista de estudantes a partir do último bimestre
        estud_list1 = df[bim_list[-1]].index.to_list()
        
        # Lista de disciplinas a partir do último bimestre
        disc = df[bim_list[-1]].columns.get_level_values(0).unique().tolist()
    else:
        # Lista de estudantes a partir do bimestre especificado
        estud_list1 = df[Bim].index.to_list()
        
        # Lista de disciplinas a partir do bimestre especificado
        disc = df[Bim].columns.get_level_values(0).unique().tolist()
    
    global estud_list
    estud_list = estud_list1
    
    global disc_list
    disc_list = disc

def showsheet(frame, dfxx):
    c0 = dfxx.columns.get_level_values(2).tolist()  # Obtendo a terceira camada de níveis das colunas
    c1 = dfxx.columns.get_level_values(1).tolist()  # Obtendo a segunda camada de níveis das colunas
    c2 = dfxx.columns.get_level_values(0).tolist()  # Obtendo a primeira camada de níveis das colunas
    col = []
    for x in range(len(c2)):
        col.append(f'{c0[x]} \n{c1[x]} \n{c2[x]}')  # Criando uma lista de rótulos de colunas formatados

    dfxx = pd.concat([pd.DataFrame([dfxx.columns.get_level_values(1)], columns=dfxx.columns), dfxx], ignore_index=True)
    dfxx = pd.concat([pd.DataFrame([dfxx.columns.get_level_values(0)], columns=dfxx.columns), dfxx], ignore_index=True)
    dfxx = pd.concat([pd.DataFrame([col], columns=dfxx.columns), dfxx], ignore_index=True)  # Concatenando rótulos de colunas no DataFrame

    lst_data = dfxx.values.tolist()  # Convertendo o DataFrame para uma lista de listas
    headers = col  # Definindo os rótulos de colunas para uso posterior

    sheet = Sheet(frame, data=lst_data)  # Criando um widget Sheet com os dados fornecidos
    sheet.enable_bindings()
    sheet.highlight_rows(rows=[0, 1, 2], bg="#238E23", fg="white")  # Destacando as primeiras 3 linhas com cores de fundo e texto específicas
    sheet.column_width(column=0, width=250)  # Definindo a largura da primeira coluna
    sheet.row_height(row=1, height="3")  # Definindo a altura da segunda linha

    sheet.change_theme(theme="dark blue", redraw=True)  # Alterando o tema da Sheet para "dark blue"
    sheet.headers(newheaders=0, index=None, reset_col_positions=False, show_headers_if_not_sheet=True, redraw=False)  # Configurando os rótulos de colunas
    sheet.row_index(newindex=0, index=None, reset_row_positions=True, show_index_if_not_sheet=True, redraw=True)  # Configurando o índice de linhas
    sheet.font(newfont=("Arial", 10, "normal"), reset_row_positions=True)  # Configurando a fonte das células
    sheet.align(align="w", redraw=True)  # Alinhando o conteúdo das células à esquerda
    sheet.row_index_align(align="w", redraw=True)  # Alinhando o índice de linhas à esquerda
    sheet.default_header_height(height=80)  # Definindo a altura padrão dos cabeçalhos de coluna
    sheet.default_row_height(height="3")  # Definindo a altura padrão das linhas
    sheet.grid(row=0, column=0, sticky="nswe")  # Posicionando o widget Sheet dentro do frame


def Selecionar(*args):
    turma = SelecTurma.get()  # Obtendo a opção selecionada para turma
    Bim = SelecBim.get()  # Obtendo a opção selecionada para bimestre
    
    selListDiscEstud(turma, Bim)  # Chamando a função para selecionar as listas de estudantes e disciplinas
    
    SelecEstud.configure(values=["Todos"] + estud_list)  # Configurando as opções disponíveis para seleção de estudantes
    SelecEstud.set("Todos")  # Definindo o valor padrão para seleção de estudantes como "Todos"
    
    SelecDisc.configure(values=["Todas"] + disc_list)  # Configurando as opções disponíveis para seleção de disciplinas
    SelecDisc.set("Todas")  # Definindo o valor padrão para seleção de disciplinas como "Todas"
    

def Selecionar2(*args):
    turma = SelecTurma.get()  # Obtendo a opção selecionada para turma
    Bim = SelecBim.get()  # Obtendo a opção selecionada para bimestre
    
    if Bim == "Todos":
        Bim = bim_list[:]  # Definindo todos os bimestres como opção selecionada
    else:
        Bim = SelecBim.get()
        
    Disc = SelecDisc.get()  # Obtendo a opção selecionada para disciplina
    if Disc == "Todas":
        Disc = disc_list[:]  # Definindo todas as disciplinas como opção selecionada
    else:
        Disc = SelecDisc.get()

    estud = SelecEstud.get()  # Obtendo a opção selecionada para estudante
    if estud == "Todos":
        estud = estud_list[:]  # Definindo todos os estudantes como opção selecionada
        idx = pd.IndexSlice
        df2 = df.loc[idx[estud], idx[Bim, Disc, :]]
        df2 = pd.DataFrame(df2)
    else:
        estud = SelecEstud.get()
        idx = pd.IndexSlice
        df2 = df.loc[idx[estud], idx[Bim, Disc, :]]
        df2 = pd.DataFrame(np.array([df2.values.tolist()]), columns=df2.index, index=[estud])

    df2index = df2.reset_index()  # Redefinindo o índice do DataFrame
    df2index = df2index.rename(columns={'index': 'ESTUDANTE'})  # Renomeando a coluna do índice como 'ESTUDANTE'
    
    showsheet(frame_3, df2index)  # Chamando a função para exibir o DataFrame em um widget Sheet

##############  FRAMES ############################
frame_1 = ctk.CTkFrame(master=App)
frame_1.grid(row=0, column=0, padx=(10, 10), pady=(10, 10), sticky="nsew")

frame_2 = ctk.CTkFrame(master=App)
frame_2.grid(row=0, column=3, padx=(10, 10), pady=(10, 10), sticky="nsew")
frame_2.columnconfigure(0, weight=100)
frame_2.rowconfigure(0, weight=100)

frame_3 = ctk.CTkFrame(master=App)
frame_3.grid(row=1, column=0, padx=(10, 10), pady=(10, 10), columnspan=4, rowspan=1, sticky="nsew")
frame_3.columnconfigure(0, weight=100)
frame_3.rowconfigure(0, weight=100)

frame_5 = ctk.CTkFrame(master=App)
frame_5.grid(row=0, column=1, padx=(10, 10), pady=(10, 10), sticky="nsew")

frame_6 = ctk.CTkFrame(master=App)
frame_6.grid(row=0, column=2, padx=(10, 10), pady=(10, 10), sticky="nsew")
frame_6.columnconfigure(0, weight=100)
frame_6.rowconfigure(0, weight=100)

App.columnconfigure(0, weight=100)
App.columnconfigure(1, weight=100)
App.columnconfigure(2, weight=1)
App.columnconfigure(3, weight=1)
App.rowconfigure(0, weight=1)
App.rowconfigure(1, weight=100)

### TURMAS ###
# Texto:
texTurma = ctk.CTkLabel(master=frame_1, text="TURMA:", font=('Arial', 16, 'bold'))
texTurma.grid(column=0, row=0, padx=5, pady=5)

# Lista de turmas como uma variável:
Turma_var = ctk.StringVar(value=turma_list[0])  # Define o valor inicial
SelecTurma = ctk.CTkOptionMenu(master=frame_1, values=turma_list, font=('Arial', 16, 'bold'), variable=Turma_var,
                               command=Selecionar)  # Cria o menu de opções com as turmas
SelecTurma.grid(column=1, row=0, padx=5, pady=5)

### BIMESTRE ###
# Texto Bimestre
texBim = ctk.CTkLabel(master=frame_1, text="BIMESTRE:", font=('Arial', 16, 'bold'))
texBim.grid(column=0, row=2, padx=5, pady=5)

Bim_var = ctk.StringVar(value=bim_list[0])
SelecBim = ctk.CTkOptionMenu(master=frame_1, values=["Todos"] + bim_list, font=('Arial', 16, 'bold'),
                             variable=Bim_var, command=Selecionar)
SelecBim.grid(column=1, row=2, padx=5, pady=5)

# Selecionando valor inicial da turma e do bimestre
selListDiscEstud(turma_list[0], bim_list[0])

####### DISCIPLINAS ###############
# Texto:
textexDisc = ctk.CTkLabel(master=frame_5, text="DISCIPLINA:", font=('Arial', 16, 'bold'))
textexDisc.grid(column=0, row=0, padx=5, pady=5)

Disc_var = ctk.StringVar(value=disc_list[0])  # Define o valor inicial
SelecDisc = ctk.CTkOptionMenu(master=frame_5, values=["Todas"] + disc_list, font=('Arial', 16, 'bold'),
                              variable=Disc_var)  # Cria o menu de opções com as disciplinas
SelecDisc.grid(column=1, row=0, padx=5, pady=5)

### ESTUDANTE ###
# Texto estudante
texEstud = ctk.CTkLabel(master=frame_5, text="ESTUDANTE:", font=('Arial', 16, 'bold'))
texEstud.grid(column=0, row=2, padx=5, pady=5)

# Lista de seleção do Estudante:
Estud_var = ctk.StringVar(value="Todos")
SelecEstud = ctk.CTkOptionMenu(master=frame_5, values=["Todos"] + estud_list, font=('Arial', 16, 'bold'),
                              variable=Estud_var)
SelecEstud.grid(column=1, row=2, padx=5, pady=5)

# Fazendo a primeira execução do código:
Selecionar()
Selecionar2()

######### BOTÃO Selecionar  #######
botexit = ctk.CTkButton(master=frame_6, text='SELECIONAR', font=('Arial', 16, 'bold'),
                        command=Selecionar2)
botexit.grid(column=0, row=0, padx=5, pady=5, sticky="nsew")

######### BOTÃO SAIR  #######
botexit = ctk.CTkButton(master=frame_2, text='SAIR', font=('Arial', 16, 'bold'),
                        command=lambda: App.destroy())
botexit.grid(column=0, row=0, padx=5, pady=5, sticky="nsew")

App.mainloop()